In [ ]:
import re
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import psycopg2
from config import db_init
from decimal import Decimal

# [SETTING DISPLAY OPTION DATAFRAME]
pd.set_option("display.max_rows", 100, "display.max_columns", 100)

# [VARIABLES DB]
filename = 'db_dwh.ini'
section = 'db'
db_info = db_init(filename, section)

# [CONNECTION & CURSOR]
db_connection = psycopg2.connect(**db_info)
db_cursor = db_connection.cursor()

"""
    GET ALL TK KEMANDORAN PANEN, LOADING, PERAWATAN, SUPERVISI
"""
query1 = 'select "ESTATE", "DEVICE ID", "DEVICE NAME", "DEVICE CODE" from "L1_Fact_AWS_ARS";'
db_cursor.execute(query1)
df_m_aws_ars = pd.DataFrame(
    db_cursor.fetchall(),
    columns=["ESTATE", "DEVICE ID", "DEVICE NAME", "DEVICE CODE"]
)

list_Device_ID = list(df_m_aws_ars["DEVICE ID"])
list_device_code = list(df_m_aws_ars["DEVICE CODE"])
# df_map_deviceName = df_m_aws_ars[['DEVICE ID', 'DEVICE NAME']]
# dict_device_name = dict(
#         zip(
#             df_map_deviceName['DEVICE ID'],
#             df_map_deviceName['DEVICE NAME']
#         )
#     )
def API_retrieved_aws_ars():

    endOfDate = datetime.today().date()
    startOfDate = datetime.today().date() - timedelta(days=7)
    print("Start Of Date: " + str(startOfDate) + "\n" + "Before Of Date: " + str(endOfDate))

    for d in list_Device_ID:

        r = requests.get(
            "http://forwarding.mertani.my.id/pull-sensor-record?deviceId="+d+"&fromDate="+str(startOfDate)+"&endDate"
            "="+str(endOfDate)+"&zone=0",
            headers={"Token": "xxxxxxxx-xxxx-xxxx-xxxxxxxxxxxx"}
        )

        data = json.loads(r.text)
        if "data" in data:
            if bool(data["data"]):
                print(d)
                df_aws_ars = pd.DataFrame(data["data"])
                df_aws_ars.reset_index(drop=True, inplace=True)
                df_aws_ars['unixTime'] = pd.to_datetime(df_aws_ars['unixTime'], unit='s')
                # df_aws_ars.insert(0, "ModifyDateTime", datetime.today().date())
                df_aws_ars['ModifyDateTime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                df_aws_ars.rename(columns=lambda x: x.split(' ')[0], inplace=True)
                df_cols_1 = df_aws_ars[['devId', 'lat_1', 'long_1', 'batt_1']]
                cols = df_aws_ars.columns.values
                new_cols = [re.split(r'[_]', item)[-1] for item in cols]
                df_aws_ars.columns = new_cols
                df_aws_ars.drop(['1'], axis=1, inplace=True)
                cols_without_1 = df_cols_1.columns.values

                new_cols_1 = [re.split(r'[_]', item)[0] for item in cols_without_1]
                df_cols_1.columns = new_cols_1
                df_cols_1.rename(
                    columns={'devId': 'DeviceID', 'lat': 'Latitude', 'long': 'Longitude', 'batt': 'Battery'},
                    inplace=True
                )
                df_aws_ars.rename(
                    columns={'devId': 'DeviceID', 'unixTime': 'FullDate', 'name': 'DeviceName', 'sig': 'Signal_Val', 'slrRad': 'SolarRad_Val', 'winDir': 'WindDir_Val', 'arHum': 'AirHmd_Val', 'winSpe': 'WindSpd_Val', 'arPre': 'AirPrs_Val', 'par': 'PhotoActRad_Val', 'batt': 'Battery_Val', 'arTem': 'AirTem_Val', 'rnFal': 'RainFal_Val'},
                    inplace=True
                )

                df_cols_1[[
                    'DeviceID', 'Latitude', 'Longitude', 'Battery'
                ]] = df_cols_1[[
                    'DeviceID', 'Latitude', 'Longitude', 'Battery'
                ]].astype(str)

                df_aws_ars[[
                    'Signal_Val', 'SolarRad_Val', 'WindDir_Val', 'AirHmd_Val',
                    'WindSpd_Val', 'AirPrs_Val', 'PhotoActRad_Val', 'Battery_Val', 'AirTem_Val', 'RainFal_Val'
                ]] = df_aws_ars[[
                    'Signal_Val', 'SolarRad_Val', 'WindDir_Val', 'AirHmd_Val',
                    'WindSpd_Val', 'AirPrs_Val', 'PhotoActRad_Val', 'Battery_Val', 'AirTem_Val', 'RainFal_Val'
                ]].applymap(str)

                
                list_df_aws = [df_cols_1, df_aws_ars]
                m_df_aws_ars = pd.concat(list_df_aws)
                print(m_df_aws_ars)
                breakpoint()



                m_df_aws_ars[[
                    'Longitude', 'Latitude', 'Battery', 'Signal_Val', 'SolarRad_Val', 'WindDir_Val', 'AirHmd_Val',
                    'WindSpd_Val', 'AirPrs_Val', 'PhotoActRad_Val', 'Battery_Val', 'AirTem_Val', 'RainFal_Val'
                ]] = m_df_aws_ars[[
                    'Longitude', 'Latitude', 'Battery', 'Signal_Val', 'SolarRad_Val', 'WindDir_Val', 'AirHmd_Val',
                    'WindSpd_Val', 'AirPrs_Val', 'PhotoActRad_Val', 'Battery_Val', 'AirTem_Val', 'RainFal_Val'
                ]].replace({'nan': 0})

                m_df_aws_ars = m_df_aws_ars.replace({np.nan: None})

                m_df_aws_ars_2 = m_df_aws_ars[['DeviceID', 'DeviceName', 'FullDate', 'Longitude', 'Latitude', 'Battery',
                                             'Signal_Val', 'SolarRad_Val', 'WindDir_Val', 'AirHmd_Val',
                                             'WindSpd_Val', 'AirPrs_Val', 'PhotoActRad_Val', 'Battery_Val',
                                             'AirTem_Val', 'RainFal_Val', 'ModifyDateTime']]

                # print(m_df_aws_ars_2.info())
                # print(m_df_aws_ars_2)

                tuple_aws_ars = tuple(map(tuple, m_df_aws_ars_2.values))
                print(tuple_aws_ars)
                query_insert_raw = \
                    "insert into \"L1_AWS_ARS_RAW\" (\"DeviceID\", \"DeviceName\", \"FullDate\", \"Longitude\", \"Latitude\", \"Battery\", " \
                    "\"Signal_Val\", \"SolarRad_Val\", \"WindDir_Val\", \"AirHmd_Val\", \"WindSpd_Val\", \"AirPrs_Val\", \"PhotoActRad_Val\", " \
                    "\"Battery_Val\", \"AirTem_Val\", \"RainFal_Val\", \"ModifyDateTime\") VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s," \
                    "%s,%s,%s,%s,%s,%s)"
                # db_cursor.fast_executemany = True

                exec
                db_cursor.executemany(
                    query_insert_raw,
                    tuple_aws_ars
                )
                db_connection.commit()

                # with pd.ExcelWriter("Data Master AWS ARS " + d + datetime.now().strftime("%m-%d-%Y") + ".xlsx") as writer:
                #     df_aws_ars.to_excel(writer, sheet_name="AWS-ARS")
            else:
                print("Data is empty in "+d)
        else:
            print("There is no DATA in Device: "+d)

    # return df_aws_ars


API_retrieved_aws_ars()

# print(df_aws_ars.values)
# cols1 = df_aws_ars.loc[:, ['00JM_sig'][]].tolist()
# df_aws_ars['00JM_sig'] = df_aws_ars['00JM_sig'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-sig Signal'), ''))
# df_aws_ars['00JM_slrRad'] = df_aws_ars['00JM_slrRad'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-slrRad Solar Radiation'), ''))
# df_aws_ars['00JM_winDir'] = df_aws_ars['00JM_winDir'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-winDir Wind Direction'), ''))
# df_aws_ars['00JM_arHum'] = df_aws_ars['00JM_arHum'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-arHum Air Humidity'), ''))
# df_aws_ars['00JM_winSpe'] = df_aws_ars['00JM_winSpe'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-winSpe Wind Speed'), ''))
# df_aws_ars['00JM_arPre'] = df_aws_ars['00JM_arPre'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-arPre Air Pressure'), ''))
# df_aws_ars['00JM_par'] = df_aws_ars['00JM_par'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-par Photosynthetically Active Radiation'),
#                    ''))
# df_aws_ars['00JM_batt'] = df_aws_ars['00JM_batt'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-batt Battery'), ''))
# df_aws_ars['00JM_arTem'] = df_aws_ars['00JM_arTem'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-arTem Air Temperature'), ''))
# df_aws_ars['00JM_rnFal'] = df_aws_ars['00JM_rnFal'].apply(
#     lambda x: next((y['value'] for y in x if y['alias_name'] == '00JM-rnFal Rain Fall'), ''))
# print(df_aws_ars)


# def combine_excel_per_sheet():
#     with pd.ExcelWriter("Data Master AWS ARS " + datetime.now().strftime("%m-%d-%Y") + ".xlsx") as writer:
#         API_retrieved_aws_ars().to_excel(writer, sheet_name="AWS-ARS")
# APIcall_geoFence().to_excel(writer, sheet_name='geoFence')
# APIcall_DeviceInformation().to_excel(writer, sheet_name='Device_Information')
# APIcall_DeviceHistoryData().to_excel(writer, sheet_name='Device_History_Data')
# APIcall_LatestVehiclePosition().to_excel(writer, sheet_name='Latest_Vehicle_Position')


# combine_excel_per_sheet()


Start Of Date: 2023-11-16
Before Of Date: 2023-11-23
Data is empty in MTI-Q8C6ZH2CN5YF
MTI-PW61GB76NOUM
            DeviceID Latitude  Longitude Battery  \
0   MTI-PW61GB76NOUM  1.32347  118.49075       2   
1   MTI-PW61GB76NOUM  1.32347  118.49075       2   
2   MTI-PW61GB76NOUM  1.32347  118.49075       2   
3   MTI-PW61GB76NOUM  1.32347  118.49075       2   
4   MTI-PW61GB76NOUM  1.32347  118.49075       2   
..               ...      ...        ...     ...   
54  MTI-PW61GB76NOUM      NaN        NaN     NaN   
55  MTI-PW61GB76NOUM      NaN        NaN     NaN   
56  MTI-PW61GB76NOUM      NaN        NaN     NaN   
57  MTI-PW61GB76NOUM      NaN        NaN     NaN   
58  MTI-PW61GB76NOUM      NaN        NaN     NaN   

                                   AirTem_Val  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3                                         NaN   
4                         

C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:62: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_aws_ars['unixTime'] = pd.to_datetime(df_aws_ars['unixTime'], unit='s')
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cols_1.rename(
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

MTI-RPZFM1HPN050
             DeviceID Latitude  Longitude Battery  \
0    MTI-RPZFM1HPN050  1.34684  118.51398     100   
1    MTI-RPZFM1HPN050  1.34684  118.51398     100   
2    MTI-RPZFM1HPN050  1.34684  118.51398     100   
3    MTI-RPZFM1HPN050  1.34684  118.51398     100   
4    MTI-RPZFM1HPN050  1.34684  118.51398     100   
..                ...      ...        ...     ...   
163  MTI-RPZFM1HPN050      NaN        NaN     NaN   
164  MTI-RPZFM1HPN050      NaN        NaN     NaN   
165  MTI-RPZFM1HPN050      NaN        NaN     NaN   
166  MTI-RPZFM1HPN050      NaN        NaN     NaN   
167  MTI-RPZFM1HPN050      NaN        NaN     NaN   

                                           Battery_Val  \
0                                                  NaN   
1                                                  NaN   
2                                                  NaN   
3                                                  NaN   
4                                                  NaN  

C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:62: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_aws_ars['unixTime'] = pd.to_datetime(df_aws_ars['unixTime'], unit='s')
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cols_1.rename(
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

MTI-QUB3OAMSSUXQ
             DeviceID Latitude  Longitude Battery  \
0    MTI-QUB3OAMSSUXQ  1.62759  118.12286     100   
1    MTI-QUB3OAMSSUXQ  1.62759  118.12286     100   
2    MTI-QUB3OAMSSUXQ  1.62759  118.12286     100   
3    MTI-QUB3OAMSSUXQ  1.62759  118.12286     100   
4    MTI-QUB3OAMSSUXQ  1.62759  118.12286     100   
..                ...      ...        ...     ...   
162  MTI-QUB3OAMSSUXQ      NaN        NaN     NaN   
163  MTI-QUB3OAMSSUXQ      NaN        NaN     NaN   
164  MTI-QUB3OAMSSUXQ      NaN        NaN     NaN   
165  MTI-QUB3OAMSSUXQ      NaN        NaN     NaN   
166  MTI-QUB3OAMSSUXQ      NaN        NaN     NaN   

                                   WindDir_Val  \
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
..                                         ...

...

C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:62: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_aws_ars['unixTime'] = pd.to_datetime(df_aws_ars['unixTime'], unit='s')
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cols_1.rename(
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

MTI-F40B24GBXV73
             DeviceID Latitude Longitude Battery  \
0    MTI-F40B24GBXV73  1.64541  118.0293     100   
1    MTI-F40B24GBXV73  1.64541  118.0293     100   
2    MTI-F40B24GBXV73  1.64541  118.0293     100   
3    MTI-F40B24GBXV73  1.64541  118.0293     100   
4    MTI-F40B24GBXV73  1.64541  118.0293     100   
..                ...      ...       ...     ...   
163  MTI-F40B24GBXV73      NaN       NaN     NaN   
164  MTI-F40B24GBXV73      NaN       NaN     NaN   
165  MTI-F40B24GBXV73      NaN       NaN     NaN   
166  MTI-F40B24GBXV73      NaN       NaN     NaN   
167  MTI-F40B24GBXV73      NaN       NaN     NaN   

                                   WindDir_Val  \
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
..                                         ...   
163  [{'

C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:62: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_aws_ars['unixTime'] = pd.to_datetime(df_aws_ars['unixTime'], unit='s')
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cols_1.rename(
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

MTI-2J19PO5NY1GN
             DeviceID Latitude  Longitude Battery  \
0    MTI-2J19PO5NY1GN  1.61489  118.12937     100   
1    MTI-2J19PO5NY1GN  1.61489  118.12937     100   
2    MTI-2J19PO5NY1GN  1.61489  118.12937     100   
3    MTI-2J19PO5NY1GN  1.61489  118.12937     100   
4    MTI-2J19PO5NY1GN  1.61489  118.12937     100   
..                ...      ...        ...     ...   
163  MTI-2J19PO5NY1GN      NaN        NaN     NaN   
164  MTI-2J19PO5NY1GN      NaN        NaN     NaN   
165  MTI-2J19PO5NY1GN      NaN        NaN     NaN   
166  MTI-2J19PO5NY1GN      NaN        NaN     NaN   
167  MTI-2J19PO5NY1GN      NaN        NaN     NaN   

                                           RainFal_Val  \
0                                                  NaN   
1                                                  NaN   
2                                                  NaN   
3                                                  NaN   
4                                                  NaN  

C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:62: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_aws_ars['unixTime'] = pd.to_datetime(df_aws_ars['unixTime'], unit='s')
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cols_1.rename(
C:\Users\fathih.ahmad\AppData\Local\Temp\ipykernel_11448\445881359.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S